In [3]:
from numpy import exp, array, random, dot
import numpy as np

In [30]:
!pip install numpy==1.22.3

     ---------------------------------------- 14.7/14.7 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.3
    Uninstalling numpy-1.23.3:
      Successfully uninstalled numpy-1.23.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.8.10 requires opencv-contrib-python, which is not installed.
caer 2.0.8 requires opencv-contrib-python, which is not installed.


In [25]:


class NeuronLayer():
    def __init__(self, number_of_neurons, number_of_inputs_per_neuron):
        self.synaptic_weights = 2 * random.random((number_of_inputs_per_neuron, number_of_neurons)) - 1


class NeuralNetwork():
    def __init__(self, layer1, layer2, layers):
        self.layer1 = layer1
        self.layer2 = layer2
        if len(layers)>0:
            self.layers = layers

    # The Sigmoid function, which describes an S shaped curve.
    # We pass the weighted sum of the inputs through this function to
    # normalise them between 0 and 1.
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    # It indicates how confident we are about the existing weight.
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # We train the neural network through a process of trial and error.
    # Adjusting the synaptic weights each time.
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations, layers):
        for iteration in range(number_of_training_iterations):
            # Pass the training set through our neural network
            output_from_layer_1, output_from_layer_2, output_from_layers = self.think(training_set_inputs, layers)

            # Calculate the error for layer 2 (The difference between the desired output
            # and the predicted output).
            layer2_error = training_set_outputs - output_from_layer_2
            layer2_delta = layer2_error * self.__sigmoid_derivative(output_from_layer_2)

            if len(layers)>0:
                layer_error = np.zeros([len(layers), 4, 4])
                layer_delta = np.zeros([len(layers), 4, 4])
                layer_error[len(layers)-1] = layer2_delta.dot(self.layer2.synaptic_weights.T)
                layer_delta[len(layers)-1] = layer_error[len(layers)-1] * self.__sigmoid_derivative(output_from_layers[len(layers)-1])
                for i in range(len(layers)-1):
                    layer_error[len(layers)-i-2] = layer_error[[len(layers)-i-1]].dot(self.layers[len(layers)-i-1].synaptic_weights.T)
                    layer_delta[len(layers)-i-2] = layer_error[[len(layers)-i-2]] * self.__sigmoid_derivative(output_from_layers[len(layers)-i-2])
                layer1_error = layer_delta[0].dot(self.layers[0].synaptic_weights.T)
                layer1_delta = layer1_error * self.__sigmoid_derivative(output_from_layer_1)


            # Calculate the error for layer 1 (By looking at the weights in layer 1,
            # we can determine by how much layer 1 contributed to the error in layer 2).
            else:
                layer1_error = layer2_delta.dot(self.layer2.synaptic_weights.T)
                layer1_delta = layer1_error * self.__sigmoid_derivative(output_from_layer_1)

            # Calculate how much to adjust the weights by
            layer1_adjustment = training_set_inputs.T.dot(layer1_delta)
            if len(layers)>0:
                layer_adjustment = np.array([])
                for i in range(len(layers)):
                    layer_adjustment = np.append(layer_adjustment, training_set_inputs.T.dot(layer_delta[i]))
            layer2_adjustment = output_from_layer_1.T.dot(layer2_delta)

            # Adjust the weights.
            self.layer1.synaptic_weights += layer1_adjustment
            if len(layers)>0:
                for i in range(len(layers)):
                    self.layers[i].synaptic_weights += layer_adjustment[i]
            self.layer2.synaptic_weights += layer2_adjustment
            

    # The neural network thinks.
    def think(self, inputs, layers):
        output_from_layer1 = self.__sigmoid(dot(inputs, self.layer1.synaptic_weights))
        output_from_layers = np.array([])
        if len(layers)>0:
            for i in range(len(layers)):
                output_temp = self.__sigmoid(dot(output_from_layer1, self.layers[i].synaptic_weights))
                output_temp = np.reshape(output_temp, (16))
                output_from_layers = np.append(output_from_layers ,output_temp)


                # output_temp = self.__sigmoid(dot(output_from_layer1, self.layers[i].synaptic_weights))
            # output_from_layers = np.append(output_from_layers , np.reshape(output_temp, (4, -1)))
            output_from_layer2 = self.__sigmoid(dot(output_from_layers[len(layers)-1], self.layer2.synaptic_weights))
            
        else: 
            output_from_layer2 = self.__sigmoid(dot(output_from_layer1, self.layer2.synaptic_weights))

        return output_from_layer1, output_from_layer2, output_from_layers


    # The neural network prints its weights
    def print_weights(self,layers):
        print("    Layer 1 (4 neurons, each with 2 inputs): ")
        print(self.layer1.synaptic_weights)
        for i in range(len(layers)):
            print(self.layers[i].synaptic_weights)
        print("    Layer 2 (1 neuron, with 4 inputs):")
        print(self.layer2.synaptic_weights)
        # print(layers)

    def print_outputs(self, training_set_inputs, layers):
        output_from_layer_1, output_from_layer_2, output_from_layers = self.think(training_set_inputs, layers)
        print("output_from_layer_1", output_from_layer_1)
        print("output_from_layers", output_from_layers.shape)
        print("output_from_layer_2", output_from_layer_2)



In [26]:

if __name__ == "__main__":

    #Seed the random number generator
    random.seed(1)

    # Create layer 1 (4 neurons, each with 2 inputs)
    layer1 = NeuronLayer(4, 2)

    # Create layer 2 (a single neuron with 4 inputs)
    layer2 = NeuronLayer(1, 4)

    layers = np.array([NeuronLayer(4,4)])
    # layers = np.array([])

    # Combine the layers to create a neural network
    neural_network = NeuralNetwork(layer1, layer2, layers)

    # print("Stage 1) Random starting synaptic weights: ")
    # neural_network.print_weights(layers)

    # The training set. We have 7 examples, each consisting of 3 input values
    # and 1 output value.
    training_set_inputs = array([[0, 1], [1, 0], [1, 1], [0, 0]])
    training_set_outputs = array([[1, 1, 1, 0]]).T

    # Train the neural network using the training set.
    # Do it 60,000 times and make small adjustments each time.
    neural_network.train(training_set_inputs, training_set_outputs, 10, layers)

    # print("Stage 2) New synaptic weights after training: ")
    # neural_network.print_weights()
    neural_network.print_outputs(training_set_inputs, layers)


    # Test the neural network with a new situation.
    print("Stage 3) Considering a new situation [1, 0] -> ?: ")
    hidden_state, output, hidden = neural_network.think(array([1, 0]), layers)
    print(output)

output_from_layer_1 [[0.34769867 0.29692157 0.36751305 0.41743211]
 [0.4774817  0.59740521 0.28323783 0.39694424]
 [0.32754614 0.38524712 0.18673631 0.32048635]
 [0.5        0.5        0.5        0.5       ]]
output_from_layers (16,)
output_from_layer_2 [[0.55443148]
 [0.62575555]
 [0.50902048]
 [0.64203374]]
Stage 3) Considering a new situation [1, 0] -> ?: 


ValueError: cannot reshape array of size 4 into shape (16,)